In [39]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

import string

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('./data/labeled.csv')

In [4]:
df.shape

(14412, 2)

In [5]:
df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [6]:
df['toxic'] = df['toxic'].apply(int)
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


In [7]:
df['toxic'].value_counts()

0    9586
1    4826
Name: toxic, dtype: int64

In [16]:
# select only toxic comments and output only text of comments
for comment in df[df['toxic'] == 1]['comment'].head():
    print(comment)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [18]:
for comment in df[df['toxic'] == 0]['comment'].head():
    print(comment)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

In [19]:
train_df, test_df = train_test_split(df, test_size=500)

In [20]:
test_df.shape

(500, 2)

In [21]:
test_df['toxic'].value_counts()

0    329
1    171
Name: toxic, dtype: int64

In [22]:
train_df['toxic'].value_counts()

0    9257
1    4655
Name: toxic, dtype: int64

## Data preparation for model

In [33]:
sentence_example = df.iloc[1]['comment'] # take text from comment with id 1
tokens = word_tokenize(sentence_example, language='russian') # split by spaces
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation] # delete punctuation
russian_stop_words = stopwords.words('russian')
tokens_without_stopwords_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words] # delete russian stopwords
snowball = SnowballStemmer(language='russian')
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stopwords_and_punctuation] # make all words lowercase and delete endings of words

In [34]:
print(f'Исходный текст: {sentence_example}')
print('-------------------')
print(f'Токены: {tokens}')
print('-------------------')
print(f'Токены без пунктуации: {tokens_without_punctuation}')
print('-------------------')
print(f'Токены без пунктуации и стопслов: {tokens_without_stopwords_and_punctuation}')
print('-------------------')
print(f'Токены после стемминга: {stemmed_tokens}')
print('-------------------')

Исходный текст: Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

-------------------
Токены: ['Хохлы', ',', 'это', 'отдушина', 'затюканого', 'россиянина', ',', 'мол', ',', 'вон', ',', 'а', 'у', 'хохлов', 'еще', 'хуже', '.', 'Если', 'бы', 'хохлов', 'не', 'было', ',', 'кисель', 'их', 'бы', 'придумал', '.']
-------------------
Токены без пунктуации: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'а', 'у', 'хохлов', 'еще', 'хуже', 'Если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']
-------------------
Токены без пунктуации и стопслов: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'хуже', 'Если', 'хохлов', 'кисель', 'придумал']
-------------------
Токены после стемминга: ['хохл', 'эт', 'отдушин', 'затюкан', 'россиянин', 'мол', 'вон', 'хохл', 'хуж', 'есл', 'хохл', 'кисел', 'придума']
-------------------


In [35]:
snowball = SnowballStemmer(language='russian')
russian_stop_words = stopwords.words('russian')


def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    '''Split sentence, remove stop words, make stemming'''
    tokens = word_tokenize(sentence, language='russian')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens


In [37]:
tokenize_sentence(sentence_example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [40]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [41]:
features = vectorizer.fit_transform(train_df['comment'])

## Model training

In [43]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df['toxic']) # train model

LogisticRegression(random_state=0)

In [44]:
model.predict(features[0]) # predict class for first comment

array([0], dtype=int64)

In [45]:
train_df['comment'].iloc[0] # 1 - negative comment, 0 - neutral

'Я так понимаю само МО к газу подключено? Проблема в газораспределительной сети?\n'

In [47]:
model_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(
        tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ('model', LogisticRegression(random_state=0))
])


In [48]:
model_pipeline.fit(train_df['comment'], train_df['toxic'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001ED8BFE8F70>)),
                ('model', LogisticRegression(random_state=0))])

In [50]:
model_pipeline.predict(["Привет, у меня все нормально"]) # 0 is negative comment

array([0], dtype=int64)

In [51]:
model_pipeline.predict(['Слушай не пойти ли тебе нафиг отсуда?'])

array([1], dtype=int64)

## Measuring metrics

In [53]:
precision_score(y_true=test_df['toxic'], y_pred=model_pipeline.predict(test_df['comment']))

0.9322033898305084

In [55]:
recall_score(y_true=test_df['toxic'], y_pred=model_pipeline.predict(test_df['comment']))

0.6432748538011696